# Lightning tour

Introduces the main ways of using Saber.

## Training

Import required modules and create the config needed for training:

In [1]:
from saber.config import Config
from saber.sequence_processor import SequenceProcessor

# Parse the config file. Note if you want to use a config other 
# than the default (found at saber/config.ini), just pass its 
# filepath as argument to Config()
config = Config() 

Saber version: 0.0.1


Using TensorFlow backend.


> Note: if Saber is called from the command line, then any command line arguments provided will overide the corresponding arguments in the config file.

Create a `SequenceProcessor` object. This object coordinates training, annotation, saving and loading of models and datasets.

In [2]:
sp = SequenceProcessor(config)

In [5]:
config.config_filepath

'/Users/johngiorgi/Documents/Masters/Class/natural_language_computing/project/Saber/saber/config.ini'

We can then load the dataset (specified by the `dataset_folder` parameter). If pretrained token embeddings were specified with `token_pretrained_embedding_filepath`, you should load them here also (uncomment the `load_embeddings()` function call).

In [ ]:
sp.load_dataset()
# sp.load_embeddings()

Lastly, we create the model we would like to use (Note: models are specified by the `model_name` parameter.)

In [ ]:
sp.create_model()

We are ready to train:

In [ ]:
sp.fit()

## Saving and loading

In the following sections we introduce the saving and loading of models.

### Saving a model

Assuming the model has already been created (see above), we can easily save our model like so

In [ ]:
path_to_saved_model = '../pretrained_models/PRGE'

In [ ]:
sp.save(path_to_saved_model)

> Currently, sp.save() will save the weights from the last training epoch by default.

### Loading a model

Lets illustrate loading a model with a new `SequenceProccesor` object.

In [ ]:
# delete our previous SequenceProccesor object (if it exists)
if 'sp' in locals(): del sp

# create a new SequenceProccesor object
sp = SequenceProcessor(config)

# load a previous model
sp.load(path_to_saved_model)

## Preforming predictions with the model

We can perform predictions on raw text with the `predict()` method:

In [ ]:
sp.annotate('''Viral-mediated noisy gene expression reveals biphasic E2f1 response to MYC Gene expression. This is a test.''', jupyter=True)

## Visulize the model

We can easily create an image depicted our model. First, install the [graphviz graph library](http://www.graphviz.org/) and the [Python interface](https://pypi.python.org/pypi/graphviz).

> More info can be found [here](https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/).

In [ ]:
# set this variable equal to your Keras model object.
model_ = sp.model.model[0]

We can either: create and save an image on our local machine,

In [ ]:
from keras.utils import plot_model
plot_model(model_, to_file='model.png', show_shapes=True, show_layer_names=True)

or, visualize it directly in the notebook

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model_, show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))